In [1]:
import pyspark

myConf=pyspark.SparkConf().set("spark.driver.bindAddress", "127.0.0.1")
#myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

#### RDD 생성

In [2]:
marks=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]
marksRdd=spark.sparkContext.parallelize(marks)

## 문제 3-1의 답
#### 이름으로 합계 계산

In [3]:
marksbyname=marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .sortByKey(False)\
    .collect()
for i in marksbyname:
    print (i[0],i[1])

'임하나' 170.0
'김하나' 180.0
'김갑돌' 180.8


## 문제 3-2의 답
#### 과목으로 합계 계산

In [4]:
marksbysubject=marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[1],float(x[2])))\
    .reduceByKey(lambda x,y:x+y)\
    .collect()
for i in marksbysubject:
    print (i[0],i[1])

'English' 252.3
'Math' 278.5


## 문제 3-3의 답
#### 이름으로 합계와 개수 계산

In [5]:
marksbyname2=marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))\
    .sortByKey(False)

In [6]:
sum_counts = marksbyname2.combineByKey(
    lambda value: (value,1),
    lambda x,value: (x[0]+value, x[1]+1),
    lambda x,y: (x[0]+y[0], x[1]+y[1])
)

In [7]:
for i in sum_counts.collect():
    print (i[0],i[1])

'임하나' (170.0, 2)
'김하나' (180.0, 2)
'김갑돌' (180.8, 2)


## 문제 3-4의 답
#### 이름으로 평균 계산

In [8]:
averageByKey = sum_counts\
    .map(lambda x: (x[0],x[1][0]/x[1][1]))\
    .collect()
for i in averageByKey:
    print(i[0],i[1])

'임하나' 85.0
'김하나' 90.0
'김갑돌' 90.4
